In [ ]:
import requests
url = 'https://apitempo.inmet.gov.br/estacao/2021-06-01/2022-05-29/A201'
resultado = requests.get(url)

In [ ]:
import json
dados_org = resultado.json()
dados_str = json.dumps(dados_org)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dados_tabla = pd.read_json(dados_str)
dados_tabla

In [ ]:
dados = dados_tabla[['DT_MEDICAO', 'HR_MEDICAO', 'TEM_INS', 'UMD_INS', 'RAD_GLO']]
dados

# Renomia o titulo do nosso dados

In [ ]:
dados.columns = ['DATA','HORARIO', 'TEMPERATURA', 'UMIDADE', 'RADIACAO']

In [ ]:
dados

In [ ]:
dados.boxplot("TEMPERATURA", by='HORARIO', figsize=(24,6))

In [ ]:
dados.boxplot("UMIDADE", by='HORARIO', figsize=(24,6))

In [ ]:
dados.boxplot("RADIACAO", by='HORARIO', figsize=(24,6))

# Verificando a suma de dados null 

In [ ]:
dados.apply(pd.isnull).sum()/dados.shape[0]

In [ ]:
dados[pd.isnull(dados['TEMPERATURA'])]

In [ ]:
dados = dados.fillna(method='ffill')

In [ ]:
dados.apply(pd.isnull).sum()/dados.shape[0]

# verificando se nosso dados são numéricos

In [ ]:
dados.dtypes

In [ ]:
dados[['TEMPERATURA']].plot(figsize=(26,6))

# Agrupando os dados por horário

In [ ]:
dados.groupby(dados.HORARIO).sum()

### Criando novos colunas para Temperatura da amahão, Umidade da amahão e Radiação da amahão na tabla  

In [ ]:
dados

### Criando novos colunas para Temperatura da amahão, Umidade da amahão e Radiação da amahão na tabla 

In [ ]:
dados['TEMP_AMANHAO'] = dados.shift(-1)['TEMPERATURA']
dados['TEMP_MES_MIX'] = dados['TEMPERATURA'].rolling(30).mean()

In [ ]:
dados['UMID_AMANHAO'] = dados.shift(-1)['UMIDADE']
dados['UMID_MES_MIX'] = dados['UMIDADE'].rolling(30).mean()

In [ ]:
dados['RADI_AMANHAO'] = dados.shift(-1)['RADIACAO']
dados['RADI_MES_MIX'] = dados['RADIACAO'].rolling(30).mean()

In [ ]:
dados['HORARIO_ATUAL'] = dados.shift(3)['HORARIO']

In [ ]:
dados

### Deletando NULL

In [ ]:
dados = dados.iloc[30:,:].copy()
dados = dados.iloc[:-1, :].copy()

In [ ]:
dados

In [ ]:
from sklearn.linear_model import Ridge
reg = Ridge(alpha=0.1, solver='auto', fit_intercept=True, max_iter=None, tol=0.001, copy_X=True)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
numerio_de_dados = len(dados)+9

In [ ]:
def criar_previsões_temp(preditores, dados, reg):
    trinar = dados.loc[:numerio_de_dados]
    teste = dados.loc[numerio_de_dados:]
    reg.fit(trinar[preditores], trinar["TEMP_AMANHAO"])
    preditores = reg.predict(teste[preditores])
    erro = mean_absolute_percentage_error(teste["TEMP_AMANHAO"],preditores)
    conjunto = pd.concat([teste['TEMP_AMANHAO'], pd.Series(preditores, index=teste.index )], axis=1)
    conjunto.columns = ['ATUAL_TEMPERATURA', 'PREVISOES_TEMPERATURA']
    conjunto[['DATA','HORARIO']] = teste[['DATA','HORARIO_ATUAL']]
    return erro, conjunto

In [ ]:
def criar_previsões_umid(preditores, dados, reg):
    trinar = dados.loc[:numerio_de_dados]
    teste = dados.loc[numerio_de_dados:]
    reg.fit(trinar[preditores], trinar["UMID_AMANHAO"])
    preditores = reg.predict(teste[preditores])
    erro = mean_absolute_percentage_error(teste["UMID_AMANHAO"],preditores)
    conjunto = pd.concat([teste["UMID_AMANHAO"], pd.Series(preditores, index=teste.index)], axis=1)
    conjunto.columns = ["ATUAL_UMIDADE", "PREVISOES_UMIDADE"]
    return erro, conjunto

In [ ]:
def criar_previsões_radi(preditores, dados, reg):
    trinar = dados.loc[:numerio_de_dados]
    teste = dados.loc[numerio_de_dados:]
    reg.fit(trinar[preditores], trinar["RADI_AMANHAO"])
    preditores = reg.predict(teste[preditores])
    erro = mean_absolute_percentage_error(teste["RADI_AMANHAO"],preditores)
    conjunto = pd.concat([teste["RADI_AMANHAO"], pd.Series(preditores, index=teste.index)], axis=1)
    conjunto.columns = ["ATUAL_RADIACAO", "PREVISOES_RADIACAO"]
    return erro, conjunto

In [ ]:
preditores = ['HORARIO', 'TEMPERATURA','UMIDADE', 'RADIACAO','TEMP_MES_MIX', 'UMID_MES_MIX','RADI_MES_MIX']

In [ ]:
temp_erro, temp_prevesao = criar_previsões_temp(preditores, dados, reg)

In [ ]:
umid_erro, umid_prevesao = criar_previsões_umid(preditores, dados, reg)

In [ ]:
radi_prevesao
 

In [ ]:
radi_erro, radi_prevesao = criar_previsões_radi(preditores, dados, reg)

In [ ]:
previsoes_dados = temp_prevesao

In [ ]:
previsoes_dados[["ATUAL_UMIDADE", "PREVISOES_UMIDADE"]] = umid_prevesao[["ATUAL_UMIDADE", "PREVISOES_UMIDADE"]]

In [ ]:
previsoes_dados

In [ ]:
previsoes_dados[["ATUAL_RADIACAO", "PREVISOES_RADIACAO"]] = radi_prevesao[["ATUAL_RADIACAO", "PREVISOES_RADIACAO"]]

In [ ]:
previsoes_dados = previsoes_dados[['DATA','HORARIO','ATUAL_TEMPERATURA','PREVISOES_TEMPERATURA','ATUAL_UMIDADE','PREVISOES_UMIDADE','ATUAL_RADIACAO','PREVISOES_RADIACAO']]

In [ ]:
umid_erro

In [ ]:
temp_erro

In [ ]:
radi_erro

In [ ]:
previsoes_dados.boxplot("PREVISOES_TEMPERATURA", by='HORARIO', figsize=(24,6))

In [ ]:
previsoes_dados.boxplot("PREVISOES_UMIDADE", by='HORARIO', figsize=(24,6))


In [ ]:
previsoes_dados.boxplot("PREVISOES_RADIACAO", by='HORARIO', figsize=(24,6))